In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pydrake.all import (AddCompliantHydroelasticProperties, AddContactMaterial,
                         AddMultibodyPlant, AddRigidHydroelasticProperties, Box,
                         ConstantVectorSource, CoulombFriction, Cylinder,
                         DeformableBodyConfig, DiagramBuilder, Filament,
                         FixedOffsetFrame, GeometryInstance,
                         IllustrationProperties, Integrator, Meshcat,
                         MeshcatVisualizer, MeshcatVisualizerParams,
                         ModelInstanceIndex, MultibodyPlantConfig, Multiplexer,
                         PdControllerGains, ProximityProperties, RevoluteJoint,
                         Rgba, RigidTransform, RotationMatrix, SceneGraphConfig,
                         Simulator, SpatialInertia)

In [ ]:
meshcat = Meshcat()

In [ ]:
def AddRoller(plant, name, position,
              radius=0.03, length=0.08, density=1000,
              friction_coeff=0.2,
              color=Rgba(0.4,0.4,0.9,1)):
    roller = plant.AddRigidBody(
        name,
        SpatialInertia.SolidCylinderWithDensity(density, radius, length, [0,0,1])
    )
    offset_frame = plant.AddFrame(FixedOffsetFrame(
        f"world_offset_frame_to_{name}",
        plant.world_body(),
        RigidTransform(RotationMatrix.MakeXRotation(np.pi/2), position)
    ))
    joint = plant.AddJoint(RevoluteJoint(
        name=f"{name}_revolute_joint",
        frame_on_parent=offset_frame,
        frame_on_child=roller.body_frame(),
        axis=[0,0,1]
    ))
    proximity_props = ProximityProperties()
    AddContactMaterial(properties=proximity_props, friction=CoulombFriction(0.2,0.2))
    AddRigidHydroelasticProperties(properties=proximity_props, resolution_hint=radius * 0.4)
    plant.RegisterCollisionGeometry(
        roller, RigidTransform(), Cylinder(radius, length), name, proximity_props
    )
    plant.RegisterVisualGeometry(
        roller, RigidTransform(), Cylinder(radius, length), name, color.rgba
    )
    plant.RegisterVisualGeometry(
        roller, RigidTransform([radius/2, 0, 0]),
        Box(radius * 0.95, radius * 0.1, length * 1.001),
        f"{name}_indicator", [1,1,1,1]
    )
    return roller, joint

In [ ]:
builder = DiagramBuilder()

plant_config = MultibodyPlantConfig()
plant_config.time_step = 1e-3

scene_graph_config = SceneGraphConfig()
scene_graph_config.default_proximity_properties.compliance_type = "rigid"

plant, scene_graph = AddMultibodyPlant(plant_config, scene_graph_config, builder)

# Add rollers
roller_radius = 0.03
belt_radius = roller_radius + 0.001
roller_distance = np.pi * belt_radius * 1.5
roller1, joint1 = AddRoller(plant, "roller1", position=[0,0,0], radius=roller_radius)
roller2, joint2 = AddRoller(plant, "roller2", position=[roller_distance,0,0], radius=roller_radius)
joint1_actuator = plant.AddJointActuator("roller1_actuator", joint1)
joint1_actuator.set_controller_gains(PdControllerGains(p=10, d=1))

# Add the belt
deformable_model = plant.mutable_deformable_model()

delta_theta = np.pi * 0.1
delta_x = delta_theta * belt_radius
belt_node_position = []
for theta in np.arange(np.pi/2, np.pi*3/2, delta_theta):
    belt_node_position.append([belt_radius * np.cos(theta), 0.0, belt_radius * np.sin(theta)])
for x in np.arange(0.0, roller_distance, delta_x):
    belt_node_position.append([x, 0.0, -belt_radius])
for theta in np.arange(-np.pi/2, np.pi/2, delta_theta):
    belt_node_position.append([belt_radius * np.cos(theta) + roller_distance, 0.0, belt_radius * np.sin(theta)])
for x in np.arange(roller_distance, 0.0, -delta_x):
    belt_node_position.append([x, 0.0, belt_radius])

belt = Filament(
    closed=True,
    node_pos=np.array(belt_node_position).T,
    cross_section=Filament.RectangularCrossSection(width=0.06, height=0.002),
    first_edge_m1=[0,1,0]
)
geometry = GeometryInstance(RigidTransform(), belt, f"belt")

illus_props = IllustrationProperties()
illus_props.AddProperty("phong", "diffuse", Rgba(0.6,0.6,0.4,1.0))
geometry.set_illustration_properties(illus_props)

proximity_props = ProximityProperties()
AddCompliantHydroelasticProperties(
    properties=proximity_props,
    hydroelastic_modulus=1e5,
    resolution_hint=0.015,
)
proximity_props.AddProperty("hydroelastic", "circumferential_resolution_hint", 0.015)
proximity_props.AddProperty("hydroelastic", "longitudinal_resolution_hint", delta_x)
proximity_props.AddProperty("material", "coulomb_friction", CoulombFriction(0.6,0.6))
geometry.set_proximity_properties(proximity_props)

config = DeformableBodyConfig()
config.set_youngs_modulus(6.2e5)
config.set_poissons_ratio(0.4999)
config.set_mass_density(1155)
config.set_mass_damping_coefficient(1.1)

unused_resolution_hint = 9999
belt_body_id = deformable_model.RegisterDeformableBody(
    geometry, config, unused_resolution_hint)

undeformed = deformable_model.GetMutableBody(belt_body_id).mutable_der_model().mutable_undeformed_state()
undeformed.set_edge_length(undeformed.get_edge_length() * 0.95)

# Add a box
a = 0.03
box = plant.AddRigidBody("box", SpatialInertia.SolidBoxWithDensity(500, a,a,a))
plant.SetDefaultFreeBodyPose(box, RigidTransform(RotationMatrix.MakeZRotation(0.2), [0.12, 0, 0.06]))
plant.RegisterCollisionGeometry(box, RigidTransform(), Box(a,a,a), "box", CoulombFriction(0.1,0.1))
plant.RegisterVisualGeometry(box, RigidTransform(), Box(a,a,a), "box", [1,0.2,0.2,1])

# Add an obstacle to stop the box
obstacle = plant.AddRigidBody("obstacle")
plant.WeldFrames(plant.world_frame(), obstacle.body_frame(), RigidTransform(RotationMatrix.MakeXRotation(np.pi/2), [0,0,0.04]))
plant.RegisterCollisionGeometry(obstacle, RigidTransform(), Cylinder(0.003, 0.1), "obstacle", CoulombFriction(0.1,0.1))
plant.RegisterVisualGeometry(obstacle, RigidTransform(), Cylinder(0.003, 0.1), "obstacle", [0.2,0.2,0.2,1])

# Finalize the multibody plant
plant.Finalize()

# Add constant speed source
speed_source = builder.AddSystem(ConstantVectorSource([np.pi]))
position_source = builder.AddSystem(Integrator(1))
builder.Connect(speed_source.get_output_port(), position_source.get_input_port())
mux = builder.AddSystem(Multiplexer(2))
builder.Connect(position_source.get_output_port(), mux.get_input_port(0))
builder.Connect(speed_source.get_output_port(), mux.get_input_port(1))
builder.Connect(
    mux.get_output_port(),
    plant.get_desired_state_input_port(ModelInstanceIndex(1))
)

# Add Meshcat visualizer
visualizer = MeshcatVisualizer.AddToBuilder(
    builder, scene_graph, meshcat, MeshcatVisualizerParams()
)

# Build and simulate
diagram = builder.Build()
simulator = Simulator(diagram)

simulator.Initialize()
simulator.set_target_realtime_rate(1.0)

visualizer.StartRecording()
simulator.AdvanceTo(5.0)
visualizer.StopRecording()
visualizer.PublishRecording()